In [2]:
import pandas as pd
import numpy as np
import cv2 

In [2]:
# Image resizing ( Dimensions of images to 200*200)
from PIL import Image
import os, sys

path = "C:/Users/Nitin/Desktop/Capilary/train/images/"

dirs = os.listdir( path )

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((119,178), Image.ANTIALIAS) # 1/3 rd of both the original dimensions
            g=f.replace('images','resized_images')
            imResize.save(g + ' resized.jpg', 'JPEG', quality=90)

resize()


In [3]:
os.chdir('C:/Users/Nitin/Desktop/Capilary/train/resized_images/')
path = "C:/Users/Nitin/Desktop/Capilary/train/resized_images/"
dirs = os.listdir( path )
x=[]
for item in dirs:
    img=cv2.imread(item)
    imgarr= np.array(img)
    x.append(imgarr.astype('float32')/255.0)
    

In [4]:
# Shuffle
import random
random.shuffle(x)

In [5]:
# Train and Test
x_train = x[0:round(0.9*len(x))]
x_test =  x[round(0.9*len(x)):len(x)]


In [6]:
x_train= np.array(x_train)
x_test= np.array(x_test)

In [7]:
np.shape(x_train)

(2723, 178, 119, 3)

In [8]:
from keras.models import Sequential, Input, load_model, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape, UpSampling2D, Conv2DTranspose

Using TensorFlow backend.


In [9]:
#define the deep learning architecture
input_img = Input(shape=(178,119,3))  # adapt this if using `channels_first` image data format
x= Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x= MaxPooling2D((2, 2))(x)
x= Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x= MaxPooling2D((2, 2))(x)
encoded= Dense(512, name='encoder')(Flatten()(x))

#decoding
x= Dense(10208)(encoded)
x= Reshape((44,29,8))(x)
x= UpSampling2D((2, 2))(x)
x= Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x= UpSampling2D((2, 2))(x)
x= Conv2D(16, (3, 3), activation='relu', padding='same')(x)
decoded= Conv2DTranspose(3, (3,4), activation='sigmoid')(x)

model= Model(input_img, decoded)
#compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 178, 119, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 178, 119, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 89, 59, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 89, 59, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 44, 29, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10208)             0         
_________________________________________________________________
encoder (Dense)              (None, 512)               5227008   
__________

In [10]:
model.fit(x_train, x_train, validation_data= (x_test, x_test), epochs= 30, batch_size= 100, shuffle= True)


Train on 2723 samples, validate on 303 samples
Epoch 1/30
2723/2723 [==============================] - 170s 63ms/step - loss: 0.4542 - val_loss: 0.3500
Epoch 2/30
2723/2723 [==============================] - 186s 68ms/step - loss: 0.3129 - val_loss: 0.3059
Epoch 3/30
2723/2723 [==============================] - 198s 73ms/step - loss: 0.2864 - val_loss: 0.2869
Epoch 4/30
2723/2723 [==============================] - 164s 60ms/step - loss: 0.2697 - val_loss: 0.2697
Epoch 5/30
2723/2723 [==============================] - 166s 61ms/step - loss: 0.2572 - val_loss: 0.2599
Epoch 6/30
2723/2723 [==============================] - 165s 61ms/step - loss: 0.2473 - val_loss: 0.2539
Epoch 7/30
2723/2723 [==============================] - 165s 61ms/step - loss: 0.2410 - val_loss: 0.2483
Epoch 8/30
2723/2723 [==============================] - 164s 60ms/step - loss: 0.2363 - val_loss: 0.2436
Epoch 9/30
2723/2723 [==============================] - 164s 60ms/step - loss: 0.2333 - val_loss: 0.2416
Epoch 10

In [11]:
encoder= Model(inputs=model.input, outputs=model.get_layer('encoder').output)

In [12]:
encoder

In [13]:
os.chdir('C:/Users/Nitin/Desktop/Capilary/train/resized_images/')
path = "C:/Users/Nitin/Desktop/Capilary/train/resized_images/"
dirs = os.listdir( path )
x=[]
for item in dirs:
    img=cv2.imread(item)
    imgarr= np.array(img)
    x.append(imgarr.astype('float32')/255.0)
    

In [14]:
x=np.array(x)

In [15]:
#predict the feature vectors of each image
y= encoder.predict(x)

In [16]:
for i in range(len(y)):
    img1=y[i]


In [17]:
from numpy.linalg import norm
norm(y[1,])

21.02024

In [18]:
cosine_sim=[]
for i in range(len(y)):
    mag_test=norm(y[i,])
    for j in range(len(y)):
        cos=abs(np.inner(y[j,],y[i,])/(norm(y[j,]*mag_test)))
        cosine_sim.append(cos)

In [19]:
np.shape(cosine_sim)

(9156676,)

In [20]:
cosine_sim = np.reshape(cosine_sim,[3026,3026])
cosine_sim = np.matrix(cosine_sim)

In [21]:
os.chdir('C:/Users/Nitin/Desktop/Capilary/train/resized_images/')
path = "C:/Users/Nitin/Desktop/Capilary/train/resized_images/"
dirs = os.listdir( path )
images_names=[]
for item in dirs:
    images_names.append(item)
        
        

In [22]:
for i in range(len(images_names)):
    images_names[i]=images_names[i].replace(' resized.jpg','')

In [23]:
df= pd.DataFrame(cosine_sim,columns=images_names)

In [24]:
df['image']=images_names

In [25]:
df.head(20)

,11139192,11139194,11139524,11139560,11139588,11139650,11141306,11141308,11141318,11141320,...,14122762,14122766,14122818,14122820,14122822,14122832,14122834,14128359,14129477,image
0,1.000000,0.540411,0.451240,0.510148,0.310507,0.450621,0.330411,0.397326,0.426154,0.465798,...,0.354722,0.398226,0.304113,0.113903,0.312579,0.369268,0.296594,0.181255,0.538098,11139192
1,0.540411,1.000000,0.225849,0.079349,0.343478,0.182765,0.498469,0.403348,0.580610,0.606987,...,0.056038,0.299994,0.161398,0.313992,0.108199,0.195023,0.060156,0.105762,0.134003,11139194
2,0.451240,0.225849,1.000000,0.562916,0.425020,0.902727,0.248813,0.289781,0.065774,0.208736,...,0.617220,0.491933,0.597462,0.382833,0.455781,0.685392,0.592192,0.278861,0.631222,11139524
3,0.510148,0.079349,0.562916,1.000000,0.379203,0.608886,0.164360,0.256186,0.125126,0.201817,...,0.484230,0.422893,0.355364,0.210983,0.373094,0.494480,0.454682,0.263318,0.688247,11139560
4,0.310507,0.343478,0.425020,0.379203,1.000000,0.372558,0.458899,0.520477,0.299434,0.570956,...,0.290458,0.466783,0.272180,0.263805,0.303238,0.309203,0.249277,0.254078,0.337898,11139588
5,0.450621,0.182765,0.902727,0.608886,0.372558,1.000000,0.223698,0.264367,0.065040,0.151424,...,0.687165,0.505486,0.623798,0.368799,0.474039,0.755719,0.644590,0.283312,0.679517,11139650
6,0.330411,0.498469,0.248813,0.164360,0.458899,0.223698,1.000000,0.841954,0.356625,0.516032,...,0.236404,0.305673,0.250172,0.173956,0.174802,0.287442,0.190324,0.060351,0.175551,11141306
7,0.397326,0.403348,0.289781,0.256186,0.520477,0.264367,0.841954,1.000000,0.358614,0.462221,...,0.253033,0.288710,0.260336,0.176272,0.179882,0.275352,0.240848,0.034816,0.274850,11141308
8,0.426154,0.580610,0.065774,0.125126,0.299434,0.065040,0.356625,0.358614,1.000000,0.587257,...,0.017711,0.375138,0.030199,0.169051,0.081355,0.000378,0.013440,0.105384,0.182890,11141318
9,0.465798,0.606987,0.208736,0.201817,0.570956,0.151424,0.516032,0.462221,0.587257,1.000000,...,0.095839,0.494979,0.022633,0.111093,0.001614,0.104438,0.030011,0.049356,0.218163,11141320


In [26]:
df.to_csv('cosine_file.csv')

In [5]:
df=pd.read_csv('C:/Users/Nitin/Desktop/Capilary/train/cosine_file.csv')

In [6]:
prods=[]
df1=df
for i in df1.drop(['image'],axis=1).columns:
    df1=df1.sort_values([i],ascending=[0])
    prods.append(df1[['image']].values[1:9])

In [7]:
len(prods)

3026

In [8]:
Final_Recommend2=pd.DataFrame(np.array(prods).reshape(3026,8))

In [9]:
Final_Recommend2['Product'] = df1.image

In [20]:
Final_Recommend2.head(5)

,0,1,2,3,4,5,6,7,Product
0,12658520,11659366,13038884,12657640,12407248,12657642,11714484,12657550,11139192
1,12657630,14122744,13017332,13017336,11407364,12657638,13064964,11659280,11139194
2,11139650,11145614,11147546,11147548,12437090,11145602,11144136,11145642,11139524
3,12658504,11659562,11659208,12657378,11714570,12658246,12360396,12658614,11139560
4,11659362,12657606,11660048,11148440,11141320,12988620,11603772,11407112,11139588


In [21]:
Final_Recommend2.to_csv('C:/Users/Nitin/Desktop/Capilary/Product_based_new.csv')